In [11]:
import os
import numpy as np

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

from test_helper import Test
import os.path

fileName = os.path.join('D:/', 'DM_LOCAL_OFERTA.CSV')

numPartitions = 100
LocalRawData = sc.textFile(fileName, numPartitions, 8)

In [12]:
# pega amostra e mostra tela
numPoints = LocalRawData.count()
print numPoints
samplePoints = LocalRawData.take(1)
print samplePoints

# começa limpeza dos dados
from pyspark.mllib.regression import LabeledPoint
import numpy as np


header = LocalRawData.take(1)[0] # o cabeçalho é a primeira linha do arquivo
print header

208114
[u'CO_LOCAL_OFERTA|NO_LOCAL_OFERTA|CO_IES|CO_UF_LOCAL_OFERTA|SGL_UF_LOCAL_OFERTA|CO_MUNICIPIO_LOCAL_OFERTA|NO_MUNICIPIO_LOCAL_OFERTA|IN_SEDE|CO_CURSO_POLO|CO_CURSO|IN_LOCAL_OFERTA_NEAD|IN_LOCAL_OFERTA_UAB|IN_LOCAL_OFERTA_REITORIA|IN_LOCAL_OFERTA_POLO|IN_LOCAL_OFERTA_UNID_ACADEMICA|DT_INICIO_FUNCIONAMENTO']
CO_LOCAL_OFERTA|NO_LOCAL_OFERTA|CO_IES|CO_UF_LOCAL_OFERTA|SGL_UF_LOCAL_OFERTA|CO_MUNICIPIO_LOCAL_OFERTA|NO_MUNICIPIO_LOCAL_OFERTA|IN_SEDE|CO_CURSO_POLO|CO_CURSO|IN_LOCAL_OFERTA_NEAD|IN_LOCAL_OFERTA_UAB|IN_LOCAL_OFERTA_REITORIA|IN_LOCAL_OFERTA_POLO|IN_LOCAL_OFERTA_UNID_ACADEMICA|DT_INICIO_FUNCIONAMENTO


In [18]:
import re
import datetime
from collections import namedtuple

Local = namedtuple('Local',headeritems) # gera a namedtuple Curso com os campos de header
print Local._fields

('CO_LOCAL_OFERTA', 'NO_LOCAL_OFERTA', 'CO_IES', 'CO_UF_LOCAL_OFERTA', 'SGL_UF_LOCAL_OFERTA', 'CO_MUNICIPIO_LOCAL_OFERTA', 'NO_MUNICIPIO_LOCAL_OFERTA', 'IN_SEDE', 'CO_CURSO_POLO', 'CO_CURSO', 'IN_LOCAL_OFERTA_NEAD', 'IN_LOCAL_OFERTA_UAB', 'IN_LOCAL_OFERTA_REITORIA', 'IN_LOCAL_OFERTA_POLO', 'IN_LOCAL_OFERTA_UNID_ACADEMICA', 'DT_INICIO_FUNCIONAMENTO')


In [14]:
def ParseLocal(rec):
    # utilizando re.split() vamos capturar nossos valores
    CO_LOCAL_OFERTA, NO_LOCAL_OFERTA, CO_IES, CO_UF_LOCAL_OFERTA, SGL_UF_LOCAL_OFERTA, CO_MUNICIPIO_LOCAL_OFERTA, NO_MUNICIPIO_LOCAL_OFERTA, IN_SEDE, CO_CURSO_POLO, CO_CURSO, IN_LOCAL_OFERTA_NEAD, IN_LOCAL_OFERTA_UAB, IN_LOCAL_OFERTA_REITORIA, IN_LOCAL_OFERTA_POLO, IN_LOCAL_OFERTA_UNID_ACADEMICA, DT_INICIO_FUNCIONAMENTO = tuple(rec.split('|'))
    

    return Local(CO_LOCAL_OFERTA, NO_LOCAL_OFERTA, CO_IES, CO_UF_LOCAL_OFERTA, SGL_UF_LOCAL_OFERTA, CO_MUNICIPIO_LOCAL_OFERTA, NO_MUNICIPIO_LOCAL_OFERTA, IN_SEDE, CO_CURSO_POLO, CO_CURSO, IN_LOCAL_OFERTA_NEAD, IN_LOCAL_OFERTA_UAB, IN_LOCAL_OFERTA_REITORIA, IN_LOCAL_OFERTA_POLO, IN_LOCAL_OFERTA_UNID_ACADEMICA, DT_INICIO_FUNCIONAMENTO)


In [15]:
# Aplique a função ParseCurso para cada objeto da base
LocalHeadlessRDD = (LocalRawData
                    .filter(lambda x: x != header)
                    .map(ParseLocal)
                    )


firstClean = LocalHeadlessRDD.take(1)[0]

In [16]:
totalRecs = LocalHeadlessRDD.count()
print firstClean
print totalRecs

Local(CO_LOCAL_OFERTA=u'105340', NO_LOCAL_OFERTA=u'Campus Universit\ufffdrio de Cuiab\ufffd', CO_IES=u'1', CO_UF_LOCAL_OFERTA=u'51', SGL_UF_LOCAL_OFERTA=u'MT', CO_MUNICIPIO_LOCAL_OFERTA=u'5103403', NO_MUNICIPIO_LOCAL_OFERTA=u'Cuiab\ufffd', IN_SEDE=u'1', CO_CURSO_POLO=u'', CO_CURSO=u'1', IN_LOCAL_OFERTA_NEAD=u'1', IN_LOCAL_OFERTA_UAB=u'1', IN_LOCAL_OFERTA_REITORIA=u'1', IN_LOCAL_OFERTA_POLO=u'1', IN_LOCAL_OFERTA_UNID_ACADEMICA=u'1', DT_INICIO_FUNCIONAMENTO=u'05/09/1952')
208113


In [17]:
# Os 20 locais com mais ofertas

LocalOfertaCountRDD = (LocalHeadlessRDD
               .map(lambda rec: (rec.NO_LOCAL_OFERTA,1))
               .reduceByKey(lambda x,y: x+y)
               )
LocalOfertaCount = sorted(LocalOfertaCountRDD.collect(), key=lambda x: -x[1])
print '\nOs 20 municipios com mais oferta de curso:\n{0}'.format(LocalOfertaCount[:20])


Os 20 municipios com mais oferta de curso:
[(u'Unidade SEDE', 4328), (u'Exclus\ufffdo Proc. 200809998 Duplicado', 1344), (u'POLO NA SEDE', 533), (u'SEDE', 407), (u'S\ufffdO PAULO', 340), (u'MARING\ufffd', 312), (u'UNIDADE SEDE', 287), (u'CAMPO GRANDE', 262), (u'SANTA MARIA', 258), (u'Cidade Universit\ufffdria', 253), (u'Campus Principal', 236), (u'BEL\ufffdM', 223), (u'NITER\ufffdI', 215), (u'SALGUEIRO', 214), (u'LAURO DE FREITAS', 210), (u'RIO GRANDE', 205), (u'DOURADOS', 203), (u'RIO DE JANEIRO', 202), (u'CURITIBA', 198), (u'BOA VISTA', 196)]
